In [ ]:
%cd /Users/Pracioppo/Desktop/VPTR/VPTR

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

import torchvision
import torchvision.transforms.functional as TF

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# from tqdm import tqdm # Loading bar
from tqdm.notebook import tqdm
from PIL import Image
import os.path
import cv2
from tabulate import tabulate
import scipy.io as sio

from pathlib import Path
import random
from datetime import datetime

from model import VPTREnc, VPTRDec, VPTRDisc, init_weights, VPTRFormerFAR
from model import GDL, MSELoss, L1Loss, GANLoss, BiPatchNCE
from utils import KTHDataset, BAIRDataset, MovingMNISTDataset
from utils import get_dataloader
from utils import visualize_batch_clips, save_ckpt, load_ckpt, set_seed, AverageMeters, init_loss_dict, write_summary, resume_training

from utils import VidCenterCrop, VidPad, VidResize, VidNormalize, VidReNormalize, VidCrop, VidRandomHorizontalFlip, VidRandomVerticalFlip, VidToTensor
from utils import set_seed, get_dataloader
from utils import f_rescale_dataset, f_Residuals, f_reshape_training_data, f_rotate_and_zoom, f_random_crop, f_rotate_and_zoom_all, f_crop_all, f_flip_all, f_augment_dataset2

import copy
import logging

set_seed(2023)

from utils import f_get_predictions, f_plot_side_by_side, f_plot_overlay, f_plot_symmetric_difference, f_plot_difference, f_plot_diff_same_t, f_save_diff, f_get_mask, f_overlap, f_diff_subsequent_t, f_diff_subsequent_t_v2, f_table, f_eval

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

device = torch.device('cuda:0')
print(f"Using {device} device")

In [ ]:
class Data(Dataset):
    pass

train_data = Data()
val_data = Data()
test_data = Data()

In [ ]:
N = 4
# N = 8
# Nv = 3
Nv = N
batch_size = N
ngpus = 1
num_workers = 1

In [ ]:
# %cd /Users/Pracioppo/Desktop/VPTR/GA_Zubens_data

# FAFs = sio.loadmat('fafs_reg.mat')["fafs_reg"]
# # torch.from_numpy(FAFs).size()

In [ ]:
%cd /Users/Pracioppo/Desktop/VPTR/GA_Zubens_data

# data = torch.zeros(660,4,2,256,256)

FAFs = torch.from_numpy(sio.loadmat('GA_data_Zubin_256_full_FAFs.mat')["FAFs"].astype(np.float32)).reshape(660,1,4,256,256)
masks = torch.from_numpy(sio.loadmat('GA_data_Zubin_256_full_masks.mat')["masks"].astype(np.float32)).reshape(660,1,4,256,256)

masks /= torch.max(masks)
FAFs /= torch.max(FAFs)

print(FAFs.size())
print(masks.size())

In [ ]:
for i in np.arange(660):
    plt.imshow(masks[i,0,0,:,:])
    plt.show()

In [ ]:
# plt.imshow(masks[1,0,0].detach().cpu().numpy())

In [ ]:
def f_find_opt_shift(x,y,pix_shift=15):

    y_s = np.roll(y,shift=-pix_shift,axis=0)
    y_s = np.roll(y_s,shift=-pix_shift,axis=1)

    sum_diff = np.zeros((2*pix_shift,2*pix_shift))
    for i in np.arange(2*pix_shift):
        for j in np.arange(2*pix_shift):
            y_ij = np.roll(y_s,shift=i,axis=0)
            y_ij = np.roll(y_ij,shift=j,axis=1)
            
            if i-pix_shift > 0:
                y_ij[:(i-pix_shift),:] = 0
            if i-pix_shift < 0:
                y_ij[(i-pix_shift):,:] = 0
            if j-pix_shift > 0:
                y_ij[:,:(j-pix_shift)] = 0
            if j-pix_shift < 0:
                y_ij[:,(j-pix_shift):] = 0
            
            sum_diff[i,j] = np.sum(1*(y_ij-x)<0)

    opt_shift = np.array((np.argmin(np.min(sum_diff,axis=1)), np.argmin(np.min(sum_diff,axis=0)))) - pix_shift
    return opt_shift

In [ ]:
def f_shift_img(img, shift):

    img_s = np.roll(img,shift=shift[0],axis=0)
    img_s = np.roll(img_s,shift=shift[1],axis=1)

    i = shift[0]
    j = shift[1]

    if i > 0:
        img_s[0:i,:] = 0
    if i < 0:
        img_s[i:,:] = 0
    if j > 0:
        img_s[:,0:j] = 0
    if j < 0:
        img_s[:,j:] = 0

    return img_s

In [ ]:
t = 1
i = 180
pix_shift = 15

masks = torch.from_numpy(sio.loadmat('GA_data_Zubin_256_full_masks.mat')["masks"].astype(np.float32)).reshape(660,1,4,256,256)

masks /= torch.max(masks)

mask_np = mask.detach().cpu().numpy()
faf_np = faf.detach().cpu().numpy()
residual_np = np.zeros_like(mask_np)

x = mask_np[0,t,:,:]
y = mask_np[0,t+1,:,:]
x_faf = faf_np[0,t,:,:]
y_faf = faf_np[0,t+1,:,:]

opt_shift = f_find_opt_shift(x,y,pix_shift=pix_shift)

y_s = f_shift_img(y, opt_shift)
# y_s += x
y_s += 1*(x>0.5)*x
y_s[np.where(y_s>1)]=1

y_faf_s = f_shift_img(y_faf, opt_shift)

residual = y_s - x
residual -= np.min(residual)
residual /= np.max(residual)
i = opt_shift[0]
j = opt_shift[1]
print(i)
print(j)
# if i > 0:
#     residual[0:i+1,:] = 0.5
# if i < 0:
#     residual[i-1:,:] = 0.5
# if j > 0:
#     residual[:,0:j+1] = 0.5
# if j < 0:
#     residual[:,j-1:] = 0.5

mask_np[0,t+1,:,:] = y_s
faf_np[0,t+1,:,:] = y_faf_s
residual_np[0,t+1,:,:] = residual

plt.imshow(x)
plt.show()
plt.imshow(y_s)
plt.show()
plt.imshow(residual)
plt.show()
plt.imshow(1*(residual>0.5))
plt.show()

In [ ]:
def f_align_data(mask,faf,i,pix_shift=15):

    mask_np = mask.detach().cpu().numpy()
    faf_np = faf.detach().cpu().numpy()
#     residual_np = np.zeros_like(mask_np)
    
    I = 0
    J = 0
    for t in np.arange(3):
        x = mask_np[0,t,:,:]
        y = mask_np[0,t+1,:,:]
        x_faf = faf_np[0,t,:,:]
        y_faf = faf_np[0,t+1,:,:]
        
        plt.imshow((y-x))
        plt.show()

        opt_shift = f_find_opt_shift(x,y,pix_shift=pix_shift)
        I += opt_shift[0]
        J += opt_shift[1]
        print(opt_shift)

        y_s = f_shift_img(y, opt_shift)
        
        # y_s += x
#         y_s += 1*(x>0.5)*x
#         y_s[np.where(y_s>1)]=1
        
        y_faf_s = f_shift_img(y_faf, opt_shift)
        
#         residual = y_s - x
#         i = opt_shift[0]
#         j = opt_shift[1]
#         if i > 0:
#             residual[0:i,:] = 0
#         if i < 0:
#             residual[i:,:] = 0
#         if j > 0:
#             residual[:,0:j] = 0
#         if j < 0:
#             residual[:,j:] = 0

        mask_np[0,t+1,:,:] = y_s
        faf_np[0,t+1,:,:] = y_faf_s
#         residual_np[0,t+1,:,:] = residual

        plt.imshow((y_s-x))
        plt.show()

#     All_Data_reg[i] = torch.from_numpy(img_np)
    mask_reg = torch.from_numpy(mask_np)
    faf_reg = torch.from_numpy(faf_np)
#     residual_reg = torch.from_numpy(faf_np)
    
    return mask_reg, faf_reg, I/3, J/3

In [ ]:
def f_align_All_data(masks, FAFs, pix_shift=15):
    masks_reg = copy.deepcopy(masks)
    fafs_reg = copy.deepcopy(FAFs)
    I_v = np.zeros(len(masks))
    J_v = np.zeros(len(masks))

#     for i in tqdm(np.arange(len(masks))):
#     for i in tqdm(np.arange(int(len(masks)/10))*10):
    for i in np.array([550, 600, 620]):
        if i == 550 or i == 600 or i == 620:
            pix_shift = 30
        else:
            pix_shift = 15
        mask = masks[i]
        faf = FAFs[i]
        mask_reg, faf_reg, I, J = f_align_data(mask,faf,i,pix_shift=pix_shift)

        masks_reg[i] = mask_reg
        fafs_reg[i] = faf_reg
        I_v[i] = I
        J_v[i] = J
        
#         if np.abs(I) > 0 or np.abs(J) > 0:
#             print(i)
        
    return masks_reg, fafs_reg, I_v, J_v

In [ ]:
masks_reg, fafs_reg, I_v, J_v = f_align_All_data(masks, FAFs, pix_shift=15)

In [ ]:
plt.plot(I_v)
plt.plot(J_v)

masks_reg.size()
fafs_reg.size()

# torch.save(masks_reg,'Y:/VPTR/GA_Zubens_data/masks_reg')
# torch.save(fafs_reg,'Y:/VPTR/GA_Zubens_data/fafs_reg')

In [ ]:
import scipy
m1 = masks_reg.detach().cpu().numpy()
m1dic = {"masks_reg": m1}
scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/masks_reg.mat", m1dic)
print("Done")

In [ ]:
m2 = fafs_reg.detach().cpu().numpy()
m2dic = {"fafs_reg": m2}
scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/fafs_reg.mat", m2dic)
print("Done")

In [ ]:
fafs_reg = torch.from_numpy(sio.loadmat('fafs_reg.mat')["fafs_reg"].astype(np.float32)).reshape(660,1,4,256,256)
masks_reg = torch.from_numpy(sio.loadmat('masks_reg.mat')["masks_reg"].astype(np.float32)).reshape(660,1,4,256,256)

masks_reg /= torch.max(masks_reg)
fafs_reg /= torch.max(fafs_reg)

In [ ]:
masks_reg2, fafs_reg2, I_v, J_v = f_align_All_data(masks_reg, fafs_reg, pix_shift=15)

# np.array([30, 240, 300, 350, 600, 620])

In [ ]:
plt.plot(I_v)
plt.plot(J_v)

masks_reg.size()
fafs_reg.size()

In [ ]:
# m1 = masks_reg2.detach().cpu().numpy()
# m1dic = {"masks_reg2": m1}
# scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/masks_reg2.mat", m1dic)
# print("Done")

# m2 = fafs_reg2.detach().cpu().numpy()
# m2dic = {"fafs_reg2": m2}
# scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/fafs_reg2.mat", m2dic)
# print("Done")

m1 = masks_reg2.detach().cpu().numpy()
m1dic = {"masks_reg3": m1}
scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/masks_reg3.mat", m1dic)
print("Done")

m2 = fafs_reg2.detach().cpu().numpy()
m2dic = {"fafs_reg3": m2}
scipy.io.savemat("C:/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/fafs_reg3.mat", m2dic)
print("Done")

In [ ]:
masks = masks_reg2
masks_add = torch.zeros_like(masks)
masks_add[:,:,0,:,:] = masks[:,:,0,:,:]
for i in tqdm(np.arange(len(masks))):
    mask = masks[i]
    mask_np = mask.detach().cpu().numpy()
    for t in np.arange(3):
        x = mask_np[0,t,:,:]
        y = mask_np[0,t+1,:,:]
        y += 1*(x>0.5)*x
        y[np.where(y>1)] = 1
        mask_np[0,t+1,:,:] = y
    masks_add[i,:,:,:,:] = torch.from_numpy(mask_np)

In [ ]:
# masks_reg3, fafs_reg3, I_v, J_v = f_align_All_data(masks_add, fafs_reg2, pix_shift=15)

In [ ]:
plt.plot(I_v)
plt.plot(J_v)

masks_reg.size()
fafs_reg.size()

In [ ]:
# import copy
# pix_shift = 15

# masks_reg = copy.deepcopy(masks)
# masks_reg2 = copy.deepcopy(masks)
# fafs_reg = copy.deepcopy(FAFs)
# fafs_reg2 = copy.deepcopy(FAFs)
# I_v = np.zeros(len(masks))
# J_v = np.zeros(len(masks))

# for i in tqdm(np.arange(len(masks))):
# # for i in tqdm(np.arange(int(len(masks)/10))*10):
#     mask = masks[i]
#     faf = FAFs[i]
#     mask_reg, faf_reg, I, J = f_align_data(mask,faf,i,pix_shift=15)
    
#     masks_reg[i] = mask_reg
#     masks_reg2[i] = mask_reg
#     fafs_reg[i] = faf_reg
#     fafs_reg2[i] = faf_reg
#     I_v[i] = I
#     J_v[i] = J
    
# #     mask_np = mask.detach().cpu().numpy()
# #     faf_np = faf.detach().cpu().numpy()
# #     mask_reg_np = mask_reg.detach().cpu().numpy()
# #     faf_reg_np = faf_reg.detach().cpu().numpy()
# #    
# #     for t in np.arange(3):
# #         x_mask = mask_np[0,t,:,:]
# #         y_mask = mask_np[0,t+1,:,:]
# # #         x_faf = faf_np[0,t,:,:]
# #         y_faf = faf_np[0,t+1,:,:]
        
# #         x_mask_reg = mask_reg_np[0,t,:,:]
# #         y_mask_reg = mask_reg_np[0,t+1,:,:]
# #         x_faf_reg = faf_reg_np[0,t,:,:]
# #         y_faf_reg = faf_reg_np[0,t+1,:,:]
        
# #         plt.imshow((y_mask-x_mask))
# #         plt.show()
# #         plt.imshow((y_mask_reg-x_mask_reg))
# #         plt.show()
    
# #     if np.abs(I_v[i]) > 10 or np.abs(J_v[i]) > 10:
# #         mask2 = masks_reg[i]
    
# #         masks_reg2, I2, J2 = f_align_data(mask2,i,pix_shift=15)
# #         masks_reg2[i] = mask_reg2
# #         I_v[i] += I2
# #         J_v[i] += J2
        
# #         for k in np.arange(3):
# #             plt.imshow(mask[0,k].detach().cpu().numpy())
# #             plt.show()

# #             plt.imshow(mask_reg2[0,k].detach().cpu().numpy())
# #             plt.show()

# plt.plot(I_v)
# plt.plot(J_v)

In [ ]:
train_num = 560
test_num = 100

FAFs = fafs_reg
masks = masks_reg

train_FAFs = FAFs[0:train_num]
train_masks = masks[0:train_num]
test_FAFs = FAFs[train_num:]
test_masks = masks[train_num:]

# train_FAFs = f_rescale_dataset(train_FAFs)
# test_FAFs = f_rescale_dataset(test_FAFs)

train_FAFs /= 255
test_FAFs /= 255
train_masks /= 255
test_masks /= 255

train_residuals = f_Residuals(train_masks)
test_residuals = f_Residuals(test_masks)

train_residuals = train_residuals - torch.min(train_residuals)
train_residuals = train_residuals/torch.max(train_residuals)

test_residuals = test_residuals - torch.min(test_residuals)
test_residuals = test_residuals/torch.max(test_residuals)

print(train_residuals.size())

# plt.imshow(train_residuals[0,0,3].detach().cpu().numpy())

In [ ]:
num_time_steps = 4
img_sz = 256

Train_Data = torch.concatenate((train_FAFs,train_masks,train_residuals),4)
Test_Data = torch.concatenate((test_FAFs,test_masks,test_residuals),4)
All_Data = torch.concatenate((Train_Data, Test_Data))

Train_Data.size()

In [ ]:
# for i in np.arange(len(All_Data)):
#     plt.imshow(All_Data[i,0,3,:,:].detach().cpu().numpy())
#     plt.show()

In [ ]:
train_data = torch.zeros(train_num,num_time_steps-1,2,img_sz,img_sz*3)
test_data = torch.zeros(test_num,num_time_steps-1,2,img_sz,img_sz*3)
all_data = torch.zeros(train_num+test_num,num_time_steps-1,2,img_sz,img_sz*3)

for i in np.arange(train_num):
    train_data[i,:,:,:,:] = f_reshape_training_data(Train_Data[i,0,:,:,:])
    
for i in np.arange(test_num):
    test_data[i,:,:,:,:] = f_reshape_training_data(Test_Data[i,0,:,:,:])
    
for i in np.arange(train_num + test_num):
    all_data[i,:,:,:,:] = f_reshape_training_data(All_Data[i,0,:,:,:])

print(train_data.size())
print(test_data.size())
print(all_data.size())

In [ ]:
def f_remove_data(data,del_idxs):

    cur_idx = 0
    ref_data = []
    for idx in del_idxs:
        data_frag = data[cur_idx:idx]
        cur_idx = idx+1
        if ref_data == []:
            ref_data = data_frag
        else:
            ref_data = torch.concatenate((ref_data,data_frag))

    data_frag = all_data[cur_idx:]
    ref_data = torch.concatenate((ref_data,data_frag))
    return ref_data

In [ ]:
# del_idxs = np.concatenate((np.arange(10)+550, np.arange(10)+600, np.arange(10)+620))
# del_idxs
# ref_data = f_remove_data(all_data,del_idxs)
# print(ref_data.size())

In [ ]:
train_data.data = ref_data[0:540]
test_data.data = ref_data[540:]

In [ ]:
train_data.data = f_fake_residual(train_data.data)
test_data.data = f_fake_residual(test_data.data)

In [ ]:
# Plot each of the sequential images for one random data example:
def f_plot_random_img(data, data_choice = []):

    if data_choice == []:
        data_choice = np.random.choice(range(len(data)), size=1)[0]

    # Construct a figure on which we will visualize the images.
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    for idx, ax in enumerate(axes.flat):
        ax.imshow(data[data_choice,idx,0,:,:])
        ax.set_title(f"Frame {idx + 1}")
        ax.axis("off")

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    for idx, ax in enumerate(axes.flat):
        ax.imshow(data[data_choice,idx,1,:,:])
        ax.set_title(f"Frame {idx + 1}")
        ax.axis("off")

In [ ]:
img = all_data[600]
img.size()

In [ ]:
# i = 10
# pix_shift = 15

# All_Data_reg = All_Data.clone()
# I_v = np.zeros(len(All_Data))
# J_v = np.zeros(len(All_Data))

# # for i in np.arange(int(len(All_Data)/10))*10:
# for i in tqdm(np.arange(len(All_Data))):
# #     print("ITER: ", i, '/ ', len(All_Data))
#     img = All_Data_reg[i]
#     img_reg, I, J = f_align_data(img,i,pix_shift=15)
#     All_Data_reg[i] = img_reg
#     I_v[i] = I
#     J_v[i] = J

# plt.plot(I_v)
# plt.plot(J_v)

img.size()